# Diving equipment store in Haute-Savoie

## Table of contents
   1. [Introduction : Business Problem](#introduction)
   2. [Data](#data)
       - [Cities and their population in Haute-Savoie](#cities)
       - [Localization of the cities](#loc)
       - [Venues in the neighborhoods](#venues)

## 1. Introduction : Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a diving equipment store. Specifically, this report will be targeted to stakeholders interested in opening a **diving & watersport equipment store** in **Haute-Savoie**, France.

Since the store is dedicated to scuba diving and watersport activities, we will first try to detect **locations with *water access* in vincinity**. In order to limit the effect of competition, it will also be necessary to ensure that **no such sports store is already established near the selected areas**.
Then, assuming that these two conditions are met, we would prefer **frequented places** with a fairly high population density.

We will use our data science powers to generate a few most promising neighborhoods based on these criteria. Advantages of each area will then be clearly expressed, so that the stakeholders can easily make a choice.

# 2. Data <a name="data"></a>

Based on our business problem, we will have to gather data such as :
 * Name and localization of all the cities in Haute-Savoie
 * Population of each city
 * Number and type of water access in the neighborhood of each city, if any
 * Number of existing sports store in the neighborhood of each city
 

Following data sources will be needed :
 * Names and poulations of the cities in Haute-Savoie will be obtained using an available table of **Wikipedia**
 * Localization of the cities will be read from a local csv file
 * Venues such as water access and sports store will be extracted using **Foursquare API**

### a. Cities and their population in Haute-Savoie <a name="cities"></a>

Let's extract the available table of Wikipedia where all the cities ('communes' in french) of Haute-Savoie are listed with also the associated population.

In [1]:
#import libraries
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)

In [2]:
# Extract table from wikipedia
from pandas.io.html import read_html
page='https://fr.wikipedia.org/wiki/Liste_des_communes_de_la_Haute-Savoie'
table=read_html(page,attrs={"class":"wikitable"})
print('Table extracted')

Table extracted


In [3]:
# Check the first rows of the obtained dataframe
communes74=table[0]
print(communes74.shape)
communes74.head()

(280, 10)


,Nom,CodeInsee,Code postal,Arrondissement,Canton,Intercommunalité,Superficie(km2),Population(dernière pop. légale),Densité(hab./km2),Modifier
0,Annecy(préfecture),74010,7400074370746007494074960,Annecy,Annecy-1Annecy-2Annecy-le-VieuxSeynod,CA du Grand Annecy,6694,126 924 (2017),1 896,NaN
1,Abondance,74001,74360,Thonon-les-Bains,Évian-les-Bains,CC Pays d'Évian Vallée d'Abondance,5884,1 439 (2017),24,NaN
2,Alby-sur-Chéran,74002,74540,Annecy,Rumilly,CA du Grand Annecy,656,2 579 (2017),393,NaN
3,Alex,74003,74290,Annecy,Faverges,CC des vallées de Thônes,1702,1 072 (2017),63,NaN
4,Allèves,74004,74540,Annecy,Rumilly,CA du Grand Annecy,881,411 (2017),47,NaN


We only need the columns with the name and the population, so let's clean the table.

In [4]:
# Remove rows with undefined postal code
communes74.dropna(subset=['Code postal'],axis=0,inplace=True)
# Remove all colums except 'Nom' and 'Population'
communes74.drop({'CodeInsee','Code postal','Arrondissement','Canton','Intercommunalité','Superficie(km2)','Densité(hab./km2)','Modifier'},axis=1,inplace=True)
# Rename the columns as 'Commune' and 'Population'
communes74.rename(columns={'Nom':'Commune','Population(dernière pop. légale)':'Population'},inplace=True)
# Remove all comments in ()
communes74['Commune']=communes74['Commune'].str.split('(').str[0]
communes74['Population']=communes74['Population'].str.split('(').str[0]
print(communes74.shape)
communes74.head()

(279, 2)


,Commune,Population
0,Annecy,126 924
1,Abondance,1 439
2,Alby-sur-Chéran,2 579
3,Alex,1 072
4,Allèves,411


Population has to be considered as a number, so let's define the type of collected data.

In [5]:
communes74.dtypes

Commune       object
Population    object
dtype: object

In [6]:
# Remove the thousand separator in the string
def remov_sep(s):
    j=""
    for x in s.split():
        j=j+str(x)
    return j

for num in range(communes74.shape[0]):
    communes74['Population'][num]=remov_sep(communes74['Population'][num])

In [7]:
communes74[['Population']]=communes74[['Population']].astype('int')
print(communes74.dtypes)
communes74.head()

Commune       object
Population     int64
dtype: object


,Commune,Population
0,Annecy,126924
1,Abondance,1439
2,Alby-sur-Chéran,2579
3,Alex,1072
4,Allèves,411


### b. Localization of the cities <a name="loc"></a>

Now let's collect also the latitude and longitude of all theses cities. For that we have to read a local csv file.

In [8]:
# The code was removed by Watson Studio for sharing.

(279, 4)


,Commune,Code Postal,Latitude,Longitude
0,Annecy,74000\n74370\n74600\n74940\n74960,45.900002,6.11667
1,Abondance,74360,46.283329,6.73333
2,Alby-sur-Chéran,74540,45.816700,6.01670
3,Alex,74290,45.883331,6.23333
4,Allèves,74540,45.750000,6.08333


Let's first remove the postal code from the previous table and then combine the 2 databases.

In [9]:
communes_latlon.drop({'Code Postal'},axis=1,inplace=True)
comm74 = pd.merge(communes74,communes_latlon,on='Commune')
print(comm74.shape)
comm74.head()

(279, 4)


,Commune,Population,Latitude,Longitude
0,Annecy,126924,45.900002,6.11667
1,Abondance,1439,46.283329,6.73333
2,Alby-sur-Chéran,2579,45.816700,6.01670
3,Alex,1072,45.883331,6.23333
4,Allèves,411,45.750000,6.08333


In order to visualize on a map the repartition of the cities, we have to import the following libraries.

In [10]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.1               |             py_0         575 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

In [31]:
# Define Latitude and Longitude of Haute-Savoie
T_lon = 6.3833
T_lat = 46.05

# Create map of Haute-Savoie
map_htesavoie = folium.Map(location = [T_lat, T_lon], zoom_start=10)

# Add markers to map
for lat, lon, comm in zip(comm74['Latitude'], comm74['Longitude'], comm74['Commune']):
    label = '{}'.format(comm)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html=False).add_to(map_htesavoie)

map_htesavoie

### c. Venues in the neighborhoods <a name="venues"></a>

Now that we have a good overview of the cities in Haute-Savoie, let's find out where are the 'water access' and sports stores around them. We will use Foursquare API to gather this information.

Foursquare credentials are defined in hidden cell below.

In [32]:
# The code was removed by Watson Studio for sharing.

In [15]:
import requests

In [16]:
# Define a function to get nearby venues
# limit of number of venues is set to 100 and radius around each location is set to 2500 m

limit = 100

def getNearbyVenues(names, latitudes, longitudes, radius=2500):
    
    venues_list=[]
    for name, lat, lon in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lon,
            radius, 
            limit)
        
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lon, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['categories'][0]['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Commune', 
                  'Commune Latitude', 
                  'Commune Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue Id']
    
    return(nearby_venues)

Let's now go over our neighborhood locations and get nearby venues

In [17]:
htesavoie_venues = getNearbyVenues(names = comm74['Commune'], latitudes = comm74['Latitude'], longitudes = comm74['Longitude'])
print(htesavoie_venues.shape)
htesavoie_venues.head()

Annecy
Abondance
Alby-sur-Chéran
Alex
Allèves
Allinges
Allonzier-la-Caille
Amancy
Ambilly
Andilly
Annemasse
Anthy-sur-Léman
Arâches-la-Frasse
Arbusigny
Archamps
Arenthon
Argonay
Armoy
Arthaz-Pont-Notre-Dame
Ayse
Ballaison
Bassy
Beaumont
Bellevaux
Bernex
Bloye
Bluffy
Boëge
Bogève
Bonne
Bonnevaux
Bonneville
Bons-en-Chablais
Bossey
Boussy
Brenthonne
Brizon
Burdignin
Cercier
Cernex
Cervens
Chainaz-les-Frasses
Challonges
Chamonix-Mont-Blanc
Champanges
Chapeiry
Charvonnex
Châtel
Châtillon-sur-Cluses
Chaumont
Chavannaz
Chavanod
Chêne-en-Semine
Chênex
Chens-sur-Léman
Chessenaz
Chevaline
Chevenoz
Chevrier
Chilly
Choisy
Clarafond-Arcine
Clermont
Cluses
Collonges-sous-Salève
Combloux
Contamine-Sarzin
Contamine-sur-Arve
Copponex
Cordon
Cornier
Cranves-Sales
Crempigny-Bonneguête
Cruseilles
Cusy
Cuvat
Demi-Quartier
Desingy
Dingy-Saint-Clair
Dingy-en-Vuache
Domancy
Doussard
Douvaine
Draillant
Droisy
Duingt
Éloise
Entrevernes
Epagny Metz-Tessy
Essert-Romand
Etaux
Étercy
Étrembières
Évian-les-Bains
Exc

,Commune,Commune Latitude,Commune Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Id
0,Annecy,45.900002,6.11667,Brumes,45.899517,6.123535,Coffee Shop,4bf58dd8d48988d1e0931735
1,Annecy,45.900002,6.11667,Beer O'Clock,45.897427,6.123039,Bar,4bf58dd8d48988d116941735
2,Annecy,45.900002,6.11667,Une Autre Histoire,45.899761,6.121999,Tea Room,4bf58dd8d48988d1dc931735
3,Annecy,45.900002,6.11667,Chez Pen,45.904349,6.121446,Bar,4bf58dd8d48988d116941735
4,Annecy,45.900002,6.11667,Le Barista Café,45.900824,6.124986,Coffee Shop,4bf58dd8d48988d1e0931735


In [21]:
htesavoie_venues.groupby('Commune').count()

,Commune Latitude,Commune Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Id
Commune,,,,,,,
Abondance,10,10,10,10,10,10,10
Alby-sur-Chéran,4,4,4,4,4,4,4
Alex,4,4,4,4,4,4,4
Allinges,5,5,5,5,5,5,5
Allonzier-la-Caille,5,5,5,5,5,5,5
Allèves,1,1,1,1,1,1,1
Amancy,9,9,9,9,9,9,9
Ambilly,55,55,55,55,55,55,55
Andilly,7,7,7,7,7,7,7


In [22]:
print('There are {} unique categories.'.format(len(htesavoie_venues['Venue Category'].unique())))

There are 228 unique categories.


We have to define the list of relevant venues. We can find all the needed documentation on Foursquare website *(https://developer.foursquare.com/docs/resources/categories)*.

In [23]:
# list of all the relevant categories
category = ['4bf58dd8d48988d193941735','4bf58dd8d48988d105941735','52e81612bcbc57f1066b7a28',
            '56aa371be4b08b9a8d573544','4bf58dd8d48988d1e2941735','52e81612bcbc57f1066b7a12',
            '4bf58dd8d48988d1e0941735','4bf58dd8d48988d160941735','4bf58dd8d48988d161941735',
            '4bf58dd8d48988d15e941735','52e81612bcbc57f1066b7a29','56aa371be4b08b9a8d573541',
            '4eb1d4dd4b900d56c88a45fd','56aa371be4b08b9a8d573560','56aa371be4b08b9a8d5734c3',
            '52e81612bcbc57f1066b7a44','52e81612bcbc57f1066b7a27','52f2ab2ebcbc57f1066b8b1a',
            '52f2ab2ebcbc57f1066b8b22','58daa1558bbb0b01f18ec1ae','4bf58dd8d48988d1ed941735',
            '4bf58dd8d48988d1f2941735','56aa371be4b08b9a8d57353e']
# sublist with only the categories related to water access
water_access_cat = ['4bf58dd8d48988d193941735','4bf58dd8d48988d105941735','52e81612bcbc57f1066b7a28',
            '56aa371be4b08b9a8d573544','4bf58dd8d48988d1e2941735','52e81612bcbc57f1066b7a12',
            '4bf58dd8d48988d1e0941735','4bf58dd8d48988d160941735','4bf58dd8d48988d161941735',
            '4bf58dd8d48988d15e941735','52e81612bcbc57f1066b7a29','56aa371be4b08b9a8d573541',
            '4eb1d4dd4b900d56c88a45fd','56aa371be4b08b9a8d573560','56aa371be4b08b9a8d5734c3',
            '52e81612bcbc57f1066b7a44','52e81612bcbc57f1066b7a27','58daa1558bbb0b01f18ec1ae',
            '4bf58dd8d48988d1ed941735','56aa371be4b08b9a8d57353e']
# sublist with only the categories related to shops
shops_cat = ['52f2ab2ebcbc57f1066b8b1a','52f2ab2ebcbc57f1066b8b22','4bf58dd8d48988d1f2941735']

Let's extract only the relevant venues for our research.

In [24]:
htesavoie_sel_venues = htesavoie_venues[htesavoie_venues['Venue Id'].isin(category)]
print(htesavoie_sel_venues.shape)

(109, 8)


In [25]:
# one hot encoding
htesavoie_onehot = pd.get_dummies(htesavoie_sel_venues[['Venue Category']], prefix="", prefix_sep="")

# add Commune column back to dataframe
htesavoie_onehot['Commune'] = htesavoie_sel_venues['Commune'] 

# move Commune column to the first column
fixed_columns = [htesavoie_onehot.columns[-1]] + list(htesavoie_onehot.columns[:-1])
htesavoie_onehot = htesavoie_onehot[fixed_columns]

print(htesavoie_onehot.shape)
htesavoie_onehot.head()

(109, 14)


,Commune,Beach,Dive Spot,Harbor / Marina,Hot Spring,Lake,Pool,Rafting,River,Spa,Sporting Goods Shop,Water Park,Waterfall,Waterfront
49,Annecy,1,0,0,0,0,0,0,0,0,0,0,0,0
53,Annecy,1,0,0,0,0,0,0,0,0,0,0,0,0
81,Annecy,1,0,0,0,0,0,0,0,0,0,0,0,0
142,Ambilly,0,0,0,0,0,0,0,0,0,1,0,0,0
192,Annemasse,0,0,0,0,0,0,0,0,0,1,0,0,0


And here we are ! Let's have a look to the different relevant venues around each city.

In [27]:
communes_table = htesavoie_onehot.groupby('Commune').sum().reset_index()
print(communes_table.shape)
communes_table.head()

(64, 14)


,Commune,Beach,Dive Spot,Harbor / Marina,Hot Spring,Lake,Pool,Rafting,River,Spa,Sporting Goods Shop,Water Park,Waterfall,Waterfront
0,Ambilly,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Annecy,3,0,0,0,0,0,0,0,0,0,0,0,0
2,Annemasse,0,0,0,0,0,0,0,0,0,1,0,0,0
3,Anthy-sur-Léman,0,0,0,0,0,0,0,0,0,1,0,0,0
4,Armoy,0,0,0,0,0,0,1,0,0,0,0,0,0


Let's merge all the data by city (relevant venues, population, latitude and longitude).

In [28]:
df=pd.merge(communes_table,comm74,on='Commune')
df.head()

,Commune,Beach,Dive Spot,Harbor / Marina,Hot Spring,Lake,Pool,Rafting,River,Spa,Sporting Goods Shop,Water Park,Waterfall,Waterfront,Population,Latitude,Longitude
0,Ambilly,0,0,0,0,0,0,0,0,0,1,0,0,0,6385,46.195200,6.22430
1,Annecy,3,0,0,0,0,0,0,0,0,0,0,0,0,126924,45.900002,6.11667
2,Annemasse,0,0,0,0,0,0,0,0,0,1,0,0,0,35712,46.200001,6.25000
3,Anthy-sur-Léman,0,0,0,0,0,0,0,0,0,1,0,0,0,2171,46.355300,6.42730
4,Armoy,0,0,0,0,0,0,1,0,0,0,0,0,0,1295,46.349998,6.51667


Let's now see all the collected data on a map.
All the cities that have water access are shown with a blue dot, and if there's a sports shop at the same time we add a red circle around.

In [30]:
# Define Latitude and Longitude of Haute-Savoie
T_lon = 6.3833
T_lat = 46.05

# Create map of Haute-Savoie
map2_htesavoie = folium.Map(location = [T_lat, T_lon], zoom_start=10)

# Add markers to map
for lat, lon, comm, shop in zip(df['Latitude'], df['Longitude'], df['Commune'], df['Sporting Goods Shop']):
    label = '{}'.format(comm)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = 'blue' if shop==0 else 'red',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html=False).add_to(map2_htesavoie)

map2_htesavoie

This concludes the data gathering phase - we're now ready to use this data for analysis to find an optimal location for a new dive equipement store !